In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\
import missingno as msno
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
final_file = pd.read_csv('data/zero_submission.csv')
final_file.rename(columns={'month': 'year_month', 'STATE': 'State'}, inplace=True)
final_file.head()

,ID,State,year_month,total_fire_size
0,0,AK,2011-01,0
1,1,AK,2011-02,0
2,2,AK,2011-03,0
3,3,AK,2011-04,0
4,4,AK,2011-05,0


In [3]:
final_file['month'] = pd.to_datetime(final_file['year_month']).dt.month
final_file['year'] = pd.to_datetime(final_file['year_month']).dt.year
final_file.drop(columns=['year_month'], inplace=True)
final_file.head(5)

,ID,State,total_fire_size,month,year
0,0,AK,0,1,2011
1,1,AK,0,2,2011
2,2,AK,0,3,2011
3,3,AK,0,4,2011
4,4,AK,0,5,2011


# Merged

In [4]:
merged = pd.read_csv('data/merged_state_data.csv')
merged.head(5)

,State,mean_elevation,Land Area (sq mi),Water Area (sq mi),Total Area (sq mi),Percentage of Federal Land,Urbanization Rate (%)
0,AK,580,570641,94743,665384,60.9%,64.9
1,AL,150,50645,1775,52420,2.7%,57.7
2,AR,200,52035,1143,53179,9.4%,55.5
3,AZ,1250,113594,396,113990,38.6%,89.3
4,CA,880,155779,7916,163695,45.4%,94.2


# Wethaer monthly state agreggate prep

In [5]:
weather_monthly = pd.read_csv('data/weather_monthly_state_aggregates.csv')
weather_monthly.head(5)

,State,year_month,PRCP,EVAP,TMIN,TMAX
0,AK,1992-05,5.36,44.17,-56.0,233.0
1,AK,1992-06,14.03,39.27,-17.0,322.0
2,AK,1992-07,16.78,30.91,28.0,300.0
3,AK,1992-08,10.46,20.14,-33.0,267.0
4,AK,1992-09,17.25,15.27,-61.0,183.0


In [6]:
weather_monthly['month'] = pd.to_datetime(weather_monthly['year_month']).dt.month
weather_monthly['year'] = pd.to_datetime(weather_monthly['year_month']).dt.year
weather_monthly.drop(columns=['year_month'], inplace=True)
weather_monthly.head(5)

,State,PRCP,EVAP,TMIN,TMAX,month,year
0,AK,5.36,44.17,-56.0,233.0,5,1992
1,AK,14.03,39.27,-17.0,322.0,6,1992
2,AK,16.78,30.91,28.0,300.0,7,1992
3,AK,10.46,20.14,-33.0,267.0,8,1992
4,AK,17.25,15.27,-61.0,183.0,9,1992


In [7]:
weather_monthly_2011_2015 = weather_monthly[(weather_monthly['year'] >= 2011) & (weather_monthly['year'] <= 2015)]
weather_monthly_2011_2015.head()

,State,PRCP,EVAP,TMIN,TMAX,month,year
95,AK,0.00,48.00,33.0,122.0,4,2011
96,AK,2.97,45.94,-33.0,294.0,5,2011
97,AK,15.40,25.82,6.0,306.0,6,2011
98,AK,16.52,24.84,6.0,272.0,7,2011
99,AK,23.00,9.21,0.0,239.0,8,2011


In [8]:
weather_monthly_2011_2015.shape

(2160, 7)

# Merged for submission

In [9]:
final_file.shape

(3000, 5)

In [ ]:
# Merge final_file and merged
final_merged = pd.merge(final_file, merged, on=['State'])

final_merged = pd.merge(final_merged, weather_monthly, on=['State', 'year', 'month'], how='outer')

final_merged.head(5)

,ID,State,month,year,mean_elevation,Land Area (sq mi),Water Area (sq mi),Total Area (sq mi),Percentage of Federal Land,Urbanization Rate (%),PRCP,EVAP,TMIN,TMAX
0,NaN,AK,5,1992,NaN,NaN,NaN,NaN,NaN,NaN,5.36,44.17,-56.0,233.0
1,NaN,AK,6,1992,NaN,NaN,NaN,NaN,NaN,NaN,14.03,39.27,-17.0,322.0
2,NaN,AK,7,1992,NaN,NaN,NaN,NaN,NaN,NaN,16.78,30.91,28.0,300.0
3,NaN,AK,8,1992,NaN,NaN,NaN,NaN,NaN,NaN,10.46,20.14,-33.0,267.0
4,NaN,AK,9,1992,NaN,NaN,NaN,NaN,NaN,NaN,17.25,15.27,-61.0,183.0


In [11]:
final_merged.shape

(12615, 14)

In [12]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
final_merged[['PRCP', 'EVAP', 'TMIN', 'TMAX']] = imputer.fit_transform(final_merged[['PRCP', 'EVAP', 'TMIN', 'TMAX']])

In [13]:
# # Fill missing values with the average for the State
# for column in ['PRCP', 'EVAP', 'TMIN', 'TMAX']:
#     final_merged[column] = final_merged.groupby('State')[column].transform(lambda x: x.fillna(x.mean()))
    
# final_merged.head(5)

In [14]:
final_merged_2011_2015 = final_merged[(final_merged['year'] >= 2011) & (final_merged['year'] <= 2015)]
final_merged_2011_2015

,ID,State,month,year,mean_elevation,Land Area (sq mi),Water Area (sq mi),Total Area (sq mi),Percentage of Federal Land,Urbanization Rate (%),PRCP,EVAP,TMIN,TMAX
95,0.0,AK,1,2011,580.0,570641.0,94743.0,665384.0,60.9%,64.9,22.792403,62.285918,11.948875,318.659788
96,1.0,AK,2,2011,580.0,570641.0,94743.0,665384.0,60.9%,64.9,22.792403,62.285918,11.948875,318.659788
97,2.0,AK,3,2011,580.0,570641.0,94743.0,665384.0,60.9%,64.9,22.792403,62.285918,11.948875,318.659788
98,3.0,AK,4,2011,580.0,570641.0,94743.0,665384.0,60.9%,64.9,0.000000,48.000000,33.000000,122.000000
99,4.0,AK,5,2011,580.0,570641.0,94743.0,665384.0,60.9%,64.9,2.970000,45.940000,-33.000000,294.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12581,2995.0,WY,8,2015,2040.0,97093.0,720.0,97813.0,46.7%,62.0,5.950000,102.050000,44.000000,344.000000
12582,2996.0,WY,9,2015,2040.0,97093.0,720.0,97813.0,46.7%,62.0,2.450000,103.080000,22.000000,367.000000
12583,2997.0,WY,10,2015,2040.0,97093.0,720.0,97813.0,46.7%,62.0,22.792403,62.285918,11.948875,318.659788
12584,2998.0,WY,11,2015,2040.0,97093.0,720.0,97813.0,46.7%,62.0,22.792403,62.285918,11.948875,318.659788


In [15]:
# import os
# import matplotlib.pyplot as plt
# import pandas as pd

# # Get the list of unique states
# states = weather_monthly['State'].unique()

# # Plot data for each state
# for state in states:
#     state_data = weather_monthly[weather_monthly['State'] == state]
    
#     # Calculate monthly averages
#     monthly_avg = state_data.groupby('month').agg({
#         'PRCP': 'mean',
#         'EVAP': 'mean',
#         'TMIN': 'mean',
#         'TMAX': 'mean'
#     }).reset_index()
    
#     # Create a figure with 4 subplots
#     fig, axes = plt.subplots(2, 2, figsize=(12, 8))
#     fig.suptitle(f'Average Weather Data for {state}', fontsize=16)
    
#     # Plot PRCP
#     axes[0, 0].bar(monthly_avg['month'], monthly_avg['PRCP'], color='blue')
#     axes[0, 0].set_title('Average Precipitation (PRCP)')
#     axes[0, 0].set_xlabel('Month')
#     axes[0, 0].set_ylabel('PRCP')
#     axes[0, 0].grid(True)
    
#     # Plot EVAP
#     axes[0, 1].bar(monthly_avg['month'], monthly_avg['EVAP'], color='green')
#     axes[0, 1].set_title('Average Evaporation (EVAP)')
#     axes[0, 1].set_xlabel('Month')
#     axes[0, 1].set_ylabel('EVAP')
#     axes[0, 1].grid(True)
    
#     # Plot TMIN
#     axes[1, 0].bar(monthly_avg['month'], monthly_avg['TMIN'], color='orange')
#     axes[1, 0].set_title('Average Minimum Temperature (TMIN)')
#     axes[1, 0].set_xlabel('Month')
#     axes[1, 0].set_ylabel('TMIN')
#     axes[1, 0].grid(True)
    
#     # Plot TMAX
#     axes[1, 1].bar(monthly_avg['month'], monthly_avg['TMAX'], color='red')
#     axes[1, 1].set_title('Average Maximum Temperature (TMAX)')
#     axes[1, 1].set_xlabel('Month')
#     axes[1, 1].set_ylabel('TMAX')
#     axes[1, 1].grid(True)
    
#     # Adjust layout
#     plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust for the suptitle
#     plt.show()

In [18]:
final_merged_2011_2015.to_csv('data/predisction_merged_2.csv', index=False)